In [1]:
import pandas as pd
import datetime

## 상대모멘텀 계산

절대모멘텀 부분에서 월별 수익율

월별로 데이터를 묶어서 월말까지 수익율 구해서 월말로 데이터 프레임 자르는 작업

월말 수익율로 pivot_table 사용하여 확인

In [2]:
def data_preprocessing(sample, ticker, base_date):
    sample["CODE"] = ticker ## 종목코드 추가
    sample = sample[sample["Date"] >= base_date][['Date', "CODE", "Adj Close"]].copy()  ##기준 일자 이후의 데이터를 복사
    sample.reset_index(inplace=True, drop=True)
    #기준 년월
    sample["STD_YM"] = sample["Date"].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d')\
        .strftime("%Y-%m"))
    sample["1M_RET"] = 0.0  ## 수익률 컬럼
    ym_keys = list(sample["STD_YM"].unique())   ## 기준년월 리스트(중복 데이터 제거)
    return sample, ym_keys


In [3]:
def create_trade_book(sample, sample_code):
    book = sample[sample_code].copy()
    book["STD_YM"] = book.index.map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d')\
        .strftime("%Y-%m"))
    for i in sample_code:
        book['p '+i] = ""
        book["r "+i] = ""
    return book

In [5]:
# 상대모멘텀 trading
def trade(book, s_codes):
    std_ym = ""
    buy_phase = False
    # 종목별로 순회
    for i in s_codes:
        print(i)
        for j in book.index:
            # 해당 종목코드 포지션 
            if book.loc[j, "p "+i] == "" and book.shift(1).loc[j, "p "+i] == "ready " + i:
                std_ym = book.loc[j, "STD_YM"]
                buy_phase = True
            # 해당 종목 코드에서 신호가 있으면 매수상태를 유지
            if book.loc[j, "p "+i] == "" and book.loc[j, "STD_YM"] == std_ym and buy_phase == True:
                book.loc[j, "p "+i] = "buy" + i

            if book.loc[j, "p "+i] == "":       ##초기화
                std_ym = None
                buy_phase = False
    return book 

In [6]:
## 손익 계산
def multi_returns(book, s_codes):
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}

    for i in book.index:
        for j in s_codes:
            if book.loc[i, "p "+j] == "buy" + j and \
            book.shift(1).loc[i, "p "+j] == "ready"+j and \
            book.shift(2).loc[i, "p "+j] == "":     #매수를 시작한 날
                buy_dict[j] = book.loc[i, j]
                print("진입일 : ", i, "종목코드 : ", j, "진입 가격 : ", buy_dict[j])
            elif book.loc[i, "p "+j] == "" and \
            book.shift(1).loc[i, "p "+j] == "buy"+j:     #매도
                sell_dict[j] = book.loc[i, j]
                # 손익 계산
                rtn = (sell_dict[j] / buy_dict[j]) - 1
                book.loc[i, "r "+j] = rtn
                print("개별 청산일 = ", i, "종목 코드 : ", j, " 진입가격 : ", buy_dict[j], \
                    "매도가격 : ", sell_dict[j], "return : ", round(rtn*100, 2))

            if book.loc[i, "p "+j]:
                buy_dict[j] = 0.0
                sell_dict[j] = 0.0

    acc_rtn = 1.0
    for i in book.index:
        rtn = 0.0
        count = 0
        for j in s_codes:
            if book.loc[i, "p "+j] == "" and book.loc[i, "p "+j] == "buy"+j:
                count += 1
                rtn += book.loc[i, "r "+j]
        if (rtn != 0.0) & (count != 0):
            acc_rtn *= (rtn/count) + 1
            print("누적 청산일 : ", i, "청산 종목 수 : ", count, \
                "청산 수익률 : ", round((rtn/count), 4), "누적 수익률 : ", round(acc_rtn, 4))
        book.loc[i, "acc_rtn"] = acc_rtn
    print("누적 수익률 : ", round(acc_rtn, 4))    